In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import generate_data as gen_d

%matplotlib inline

# Exploration

In [2]:
df = pd.read_csv("blackjack.csv")

In [3]:
df.shape

(70000, 18)

In [4]:
df.describe()

,num_decks,dealer_open,dealer_initial,dealer_hit,dealer_num_hits,dealer_final,dealer_busts,player_inital,player_hit,player_num_hits,player_final,player_busts,player_loses,draw,player_wins,strategy
count,70000.0,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000
mean,4.0,6.530743,14.557543,0.410043,0.560286,18.096514,0.178486,14.552786,0.572486,0.845100,19.892400,0.295114,0.543814,0.073129,0.383057,0.500000
std,0.0,3.153388,4.078590,0.491845,0.774488,4.351632,0.382924,4.082299,0.494721,0.899732,3.869239,0.456097,0.498080,0.260349,0.486136,0.500004
min,4.0,1.000000,4.000000,0.000000,0.000000,4.000000,0.000000,4.000000,0.000000,0.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.0,4.000000,12.000000,0.000000,0.000000,16.000000,0.000000,12.000000,0.000000,0.000000,18.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,4.0,7.000000,15.000000,0.000000,0.000000,19.000000,0.000000,15.000000,1.000000,1.000000,20.000000,0.000000,1.000000,0.000000,0.000000,0.500000
75%,4.0,10.000000,18.000000,1.000000,1.000000,21.000000,0.000000,18.000000,1.000000,1.000000,22.000000,1.000000,1.000000,0.000000,1.000000,1.000000
max,4.0,10.000000,21.000000,1.000000,6.000000,26.000000,1.000000,21.000000,1.000000,6.000000,28.000000,1.000000,1.000000,1.000000,1.000000,1.000000


# Initial exploration

In [39]:
# Simulate trials of 1000 games
def sim_trial():
    ran = gen_d.gen_df(gen_d.gen_data(num_decks=4, df_size=500, strategy=0))
    ran = ran[["player_loses", "draw", "player_wins", "strategy"]].sum()/len(ran)
    ran = pd.DataFrame(ran, ["player_loses", "draw", "player_wins", "strategy"]).T
    rec = gen_d.gen_df(gen_d.gen_data(num_decks=4, df_size=500, strategy=1))
    rec = rec[["player_loses", "draw", "player_wins", "strategy"]].sum()/len(rec)
    rec = pd.DataFrame(rec, ["player_loses", "draw", "player_wins", "strategy"]).T
    sub = pd.concat([ran, rec])
    return sub

In [46]:
def n_trials(n=10000):
    trials = sim_trial()
    for _ in range(n):
        trials = pd.concat([trials, sim_trial()])
        
    return trials

In [47]:
n_trials(n=5)

,player_loses,draw,player_wins,strategy
0,0.574,0.076,0.350,0.0
0,0.518,0.078,0.404,1.0
0,0.562,0.060,0.378,0.0
0,0.532,0.080,0.388,1.0
0,0.574,0.062,0.364,0.0
0,0.506,0.096,0.398,1.0
0,0.554,0.056,0.390,0.0
0,0.506,0.070,0.424,1.0
0,0.564,0.092,0.344,0.0
0,0.524,0.066,0.410,1.0
